In [98]:
from scipy.spatial import distance_matrix
import numpy as np
import matplotlib.pyplot as plt
import random
import math

dataset_name = 'r101'
file_path = '../Dataset/'+dataset_name+'.txt'

# Initialize lists to store vehicle and customer data
vehicle_data = []
customer_data = []

# Read data line by line
with open(file_path, 'r') as file:
    lines = file.readlines()
    
    # Parse vehicle data
    vehicle_info_start = lines.index('VEHICLE\n') + 2
    customer_info_start = lines.index('CUSTOMER\n') + 2
    vehicle_line = lines[vehicle_info_start].strip().split()
    
    # Store vehicle data as a dictionary in a list
    vehicle_data.append({
        'NUMBER': int(vehicle_line[0]),
        'CAPACITY': int(vehicle_line[1])
    })
    
    # Parse customer data
    for line in lines[customer_info_start:]:
        if line.strip():  # Skip any empty lines
            parts = line.strip().split()
            customer_data.append({
                'CUST_NO': int(parts[0]),
                'XCOORD': int(parts[1]),
                'YCOORD': int(parts[2]),
                'DEMAND': int(parts[3]),
                'READY_TIME': int(parts[4]),
                'DUE_DATE': int(parts[5]),
                'SERVICE_TIME': int(parts[6])
            })


In [99]:
class Node:
    def __init__(self, id, x, y, demand, ready_time, due_date, service_time):
        self.id = id
        self.is_depot = (id == 0)
        self.x = x
        self.y = y
        self.demand = demand
        self.ready_time = ready_time
        self.due_date = due_date
        self.service_time = service_time
        self.service_time = 10
        self.visited = False
        
class Graph:
    def __init__(self, vehicle_data, customer_data):
        self.vehicle_data = vehicle_data
        self.customer_data = customer_data
        self.nodes = []
        self.vehicle_capacity = vehicle_data[0]['CAPACITY']
        self.num_vehicles = vehicle_data[0]['NUMBER']
        self.setup_nodes()
        self.setup_distance_matrix()
        self.setup_heuristic_matrix()
        self.initialize_pheromone_matrix()
        
    def setup_heuristic_matrix(self):
        self.heuristic_matrix = 1 / self.distance_matrix
        
    def setup_nodes(self):
        for customer in self.customer_data:
            node = Node(customer['CUST_NO'], customer['XCOORD'], customer['YCOORD'], customer['DEMAND'],
                        customer['READY_TIME'], customer['DUE_DATE'], customer['SERVICE_TIME'])
            self.nodes.append(node)
    
    def setup_distance_matrix(self):
        coords = [(node.x, node.y) for node in self.nodes]
        self.distance_matrix = distance_matrix(coords, coords)
        
    def initialize_pheromone_matrix(self):
        n = len(self.nodes)
        self.pheromone_matrix = [[1.0 for _ in range(n)] for _ in range(n)]
    
    def route_distance(self, route):
        distance = 0
        for i in range(len(route) - 1):
            distance += self.distance_matrix[route[i]][route[i + 1]]
        return distance
    
    def update_pheromones(self, ant_solutions, evaporation_rate=0.5):
        # Evaporate pheromones
        for i in range(len(self.pheromone_matrix)):
            for j in range(len(self.pheromone_matrix)):
                self.pheromone_matrix[i][j] = max(self.pheromone_matrix[i][j] * (1 - evaporation_rate), 1e-5)

        
        # Deposit pheromones based on ant solutions
        total_distance = sum(self.route_distance(route) for solution in ant_solutions for route in solution)
        for route in ant_solutions:
            for k in range(len(route) - 1):
                i, j = route[k], route[k + 1]
                self.pheromone_matrix[i][j] += (1 / self.route_distance(route)) / total_distance


        

In [160]:
def aco_vrp(G, num_ants, num_iterations, alpha=1.5, beta=2.5, evaporation_rate=0.5, q=100):
    """
    Ant Colony Optimization for Vehicle Routing Problem with Time Windows.
    """
    adj_matrix = G.distance_matrix
    capacity_max = G.vehicle_capacity
    num_points = adj_matrix.shape[0]
    ph_matrix = G.pheromone_matrix  # Initial pheromone matrix
    best_solution = None
    best_cost = float('inf')

    def calculate_probabilities(current_node, unvisited, current_time, current_capacity):
        """
        Calculate probabilities of moving to the next node considering time windows and capacity constraints.
        """
        attractiveness = []
        for neighbor in unvisited:
            travel_time = G.distance_matrix[current_node][neighbor]
            arrival_time = current_time + travel_time
            if (G.nodes[neighbor].demand + current_capacity <= capacity_max
                    and arrival_time <= G.nodes[neighbor].due_date):
                tau = ph_matrix[current_node][neighbor] ** alpha
                eta = (1.0 / G.distance_matrix[current_node][neighbor]) ** beta
                attractiveness.append(tau * eta)
            else:
                attractiveness.append(0)
        total = sum(attractiveness)
        if total == 0:  # Avoid division by zero
            return np.zeros(len(unvisited))
        return np.array(attractiveness) / total

    def construct_solution():
        routes = []
        back_time = []
        for node in G.nodes:
            node.visited = False

        while any(not node.visited for node in G.nodes[1:]):  # Exclude the depot
            current_node = 0  # Start at depot
            current_time = G.nodes[current_node].ready_time
            current_capacity = 0
            route = [current_node]

            while True:
                unvisited = [i for i in range(1, len(G.nodes)) if not G.nodes[i].visited]
                if not unvisited:
                    break

                probabilities = calculate_probabilities(current_node, unvisited, current_time, current_capacity)
                if not probabilities.any():  # No valid moves
                    break

                next_node = random.choices(unvisited, weights=probabilities)[0]
                travel_time = G.distance_matrix[current_node][next_node]
                arrival_time = current_time + travel_time

                # Validate constraints
                if (current_capacity + G.nodes[next_node].demand <= capacity_max
                        and arrival_time <= G.nodes[next_node].due_date):
                    if arrival_time < G.nodes[next_node].ready_time:
                        wait_time = G.nodes[next_node].ready_time - arrival_time
                    else:
                        wait_time = 0
                    current_time = arrival_time + wait_time + G.nodes[next_node].service_time
                    current_capacity += G.nodes[next_node].demand
                    route.append(next_node)
                    G.nodes[next_node].visited = True
                    current_node = next_node
                else:
                    break  # No valid moves; end route

            # Return to depot
            current_time = current_time + G.distance_matrix[route[-1]][0]
            route.append(0)
            routes.append(route)
            back_time.append(current_time)
        if current_time > G.nodes[0].due_date:
            print("invalide route")
        return (routes,back_time)

    def calculate_total_distance(route):
        """
        Calculate the total distance of a given route using the distance matrix.
        """
        total_distance = 0
        for i in range(len(route) - 1):
            current_node = route[i]
            next_node = route[i + 1]
            total_distance += G.distance_matrix[current_node][next_node]
        return total_distance

    
    def calculate_cost(routes):
        total_cost = 0
        num_vehicles = len(routes)
        if num_vehicles > G.num_vehicles:  # Penalty for exceeding vehicle limits
            total_cost += 10000 * (num_vehicles - G.num_vehicles)
        for route in routes:
            total_cost += calculate_total_distance(route)
        total_cost = total_cost + num_vehicles*1000
        return total_cost



    def update_pheromones(all_solutions, ph_matrix, evaporation_rate=0.5):
        # Evaporate pheromones
        for i in range(len(ph_matrix)):
            for j in range(len(ph_matrix[i])):
                ph_matrix[i][j] *= (1 - evaporation_rate)
                ph_matrix[i][j] = max(ph_matrix[i][j], 1e-5)  # Prevent near-zero pheromones

        # Deposit pheromones for each solution
        for solution in all_solutions:
            routes = solution  # Each solution is a list of routes
            for route in routes:
                route_distance = calculate_total_distance(route)
                pheromone_deposit = q / max(route_distance, 1e-5)  # Prevent division by zero
                for i in range(len(route) - 1):
                    a, b = route[i], route[i + 1]
                    ph_matrix[a][b] += pheromone_deposit
                    ph_matrix[b][a] = ph_matrix[a][b]  # Ensure symmetry



    # Main ACO loop
    for iteration in range(num_iterations):
        all_solutions = []
        all_costs = []

        for _ in range(num_ants):
            solution = construct_solution()
            cost = calculate_cost(solution[0])
            all_solutions.append(solution[0])
            all_costs.append(cost)

            if cost < best_cost:
                best_cost = cost
                best_solution = solution

        # Update pheromones with all solutions
        update_pheromones(all_solutions, G.pheromone_matrix)
        print(f"Iteration {iteration + 1}: Best cost = {best_cost}, Best back time = {best_solution[1]}")
        

    return best_solution[0], best_cost, best_solution[1]

def plot_solution(graph, solution, filename):
    """
    Plot the solution of the VRP problem.
    Args:
        graph: Graph object containing node information.
        solution: List of routes (e.g., [[0, 1, 3, 0], [0, 2, 4, 0], ...]).
    """
    # Extract node coordinates
    x_coords = [node.x for node in graph.nodes]
    y_coords = [node.y for node in graph.nodes]

    # Create the plot
    plt.figure(figsize=(10, 8))

    # Plot all nodes
    plt.scatter(x_coords, y_coords, color="grey", label="Customers", marker=".", zorder=5)
    
    # Highlight the depot (node 0)
    plt.scatter(x_coords[0], y_coords[0], color="black", label="Depot", marker="D", zorder=5, s=100)

    # Annotate each node with its ID
    for node in graph.nodes:
        plt.text(node.x, node.y, f"{node.id}", fontsize=8, ha="right", va="bottom")

    # Plot each route
    for route in solution:
        route_coords_x = [x_coords[i] for i in route]
        route_coords_y = [y_coords[i] for i in route]
        plt.plot(route_coords_x, route_coords_y, label=f"Route {solution.index(route) + 1}")

    # Customize the plot
    plt.title("Vehicle Routing Problem Solution\n" + filename)
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.legend()
    plt.grid()
    plt.show()



In [ ]:
G = Graph(vehicle_data, customer_data)

print("Depot closing time: " + str(G.nodes[0].due_date))

# Run ACO
best_solution_data, best_cost,best_time = aco_vrp(
    G,
    num_ants=50, num_iterations=100, alpha=1.5, beta=1.5, evaporation_rate=0.5, q=100
)

print("\n-----------------------------------------------------------")
print("Best cost: " + str(best_cost),"\nBest back time: " + str(best_time), " \nNumber of vehicle: " + str(len(best_solution_data)))
print("-----------------------------------------------------------\n")

i = 1
for route in best_solution_data:
    print("Route " + str(i) + ": " + str(route))
    i = i + 1

# Call the plotting function with the best solution
plot_solution(G, best_solution_data, dataset_name)

C:\Users\Alexandre\AppData\Local\Temp\ipykernel_23520\3504090478.py:27: RuntimeWarning: divide by zero encountered in divide
  self.heuristic_matrix = 1 / self.distance_matrix


Depot closing time: 230
Iteration 1: Best cost = 204067.75285678878, Best back time = [219.1850372353475, 218.2484567313166, 219.0416305603426, 219.0553851381374, 195.6124969497314, 200.0553851381374, 209.37353806008852, 219.6831552862278, 216.095023109729, 208.6977156035922, 197.4138126514911, 202.24580460251534, 160.20465053408526, 176.17601498127013, 180.6836867075288, 169.20616373302047, 144.05897218255075, 190.770329614269, 161.9317652729776, 162.4682957019364, 172.80350850198275, 198.5772282859044, 131.18373020252324, 176.46309242345563, 163.0555127546399, 135.86173937932364, 106.31843003526592, 126.66106403010389, 127.55386467836128, 112.21320343559643, 135.42640687119285, 139.35533905932738, 143.86342439892263, 75.09502310972898, 93.49509756796392, 46.76955262170047, 102.8328677803526, 129.51881339845204, 92.80096617793781, 67.47220505424423, 74.03124237432849]
Iteration 2: Best cost = 104449.4571560376, Best back time = [223.56439432334852, 202.8028775489157, 219.0416305603426

In [ ]:
j = []
k = [0.5, 1, 2, 5]
l = [1, 2, 5]
for index in range(10):
    j.append(index * 10 + 10)
    index = index + 1
    
iteration_max = len(j) * len(k) * len(l)
iteration = 0
best_solution_data = []
best_cost = []
best_time = []
min = float("inf")

            
for ants in j:
    for alpha in k:
        for beta in l:
            print("-------------------------------------------------------")
            print(str(iteration + 1) + "/" + str(iteration_max) + " Iteration with this parameters: num_ants = " + str(ants) + " alpha = " + str(alpha) + " beta = " + str(beta))
            a, b, c = aco_vrp(G, num_ants=ants, num_iterations=50, alpha=alpha, beta=beta, evaporation_rate=0.5, q=100)
            best_solution_data.append(a)
            best_cost.append(b)
            best_time.append(c)
            print("Best cost: " + str(best_cost[iteration]),"\nBest back time: " + str(best_time[iteration]), " \nNumber of vehicle: " + str(len(best_solution_data[iteration])))
            iteration = iteration + 1
            
for solution in best_solution_data:
    if len(solution) < min:
        min = len(solution)

print(min)        
    

-------------------------------------------------------
1/120 Iteration with this parameters: num_ants = 10 alpha = 0.5 beta = 1
Best cost: 82537.35476510678 
Best back time: [213.095023109729, 214.11412346859095, 215.54101966249684, 223.56439432334852, 213.095023109729, 198.16552506059645, 224.56322973905222, 218.2484567313166, 202.8028775489157, 180.18033988749895, 173.37755832643194, 168.2383047297721, 194.0365690573664, 197.4138126514911, 150.97056274847714, 176.17601498127013, 160.20465053408526, 183.5967477524977, 139.01562118716424, 135.42640687119285, 139.35533905932738, 103.5599393820081, 136.86342439892263, 135.236457569829, 112.8113883008419, 88.3238075793812, 119.53672812830654, 110.89043743820395, 110.9299509312797, 79.85699678629192]  
Number of vehicle: 30
-------------------------------------------------------
2/120 Iteration with this parameters: num_ants = 10 alpha = 0.5 beta = 2
Best cost: 82336.90944321318 
Best back time: [219.0553851381374, 213.095023109729, 213.0